In [21]:
import numpy as np
import itertools

---
## User Inputs and Preliminaries

We begin by allowing the user to define parameters for the assets, being their initial prices, up, and down factors.

"time_count" refers to time steps this model takes, with time being modelled as a finite set $\mathbb{T} = \{0, 1, ..., \textnormal{T}\}$.

"risk_free_rate" simply refers to the return of risk-free assets, where a unit of money invested in it (i.e. bonds, savings) at time $t = 0$ will yield $\textnormal{R} = (1 + r)$ units of money after 1 time-step. There is the assumption that the risk-free rate will remain unchanged at all times.

"option_type" is a choice between "call" or "put".

"selected_time" is an optional parameter (otherwise zero). ** Ignore for now **

In [22]:
risk_free_rate = 1
initial_prices = [100, 100]
up_factors = [1.1, 1.1]
down_factors = [0.8, 0.9]
strike_price = 100
option_type = "call"

asset_count = len(initial_prices)
time_count = 2
selected_time = 0

print("Asset prices:", initial_prices)
print("Up:", up_factors)
print("Down:", down_factors)
print("Option type:", option_type)

Asset prices: [100, 100]
Up: [1.1, 1.1]
Down: [0.8, 0.9]
Option type: call


---
## Formula: no-arbitrage interval upper-bound $C_{\textnormal{max}}$

Where single-asset options, within the binomial model, are priced such that they achieve a "no-arbitrage" price (it is not possible to make a sure profit without risk to any capital outlay), basket options encounter an "issue" where there is no definitive no-arbitrage price, and for extremely long and complex reasons, we have a no-arbitrage price interval, defined as the region within $C_{\textnormal{min}}$ and $C_{\textnormal{max}}$.

For the sake of simplicity, we use Kedra, Libman and Steblovskaya (2022)'s explicit formula for determining the upper-bound $C_{\textnormal{max}}$ as our pricing model.

$$
C_{\textnormal{max}}(v) = \sum_{k_0 + \dots + k_m = n-k} \frac{(n-k)!}{k_0!\dots k_m!}\textnormal{P}(\mu_0)^{k_0}\dots \textnormal{P}(\mu_m)^{k_m}\textnormal{X}(\omega_1 \dots\omega_k\mu_0\dots\mu_0\dots\mu_m\dots\mu_m)
$$

Rather than tackling the formula directly, we will do so in a more procedural fashion to simplify its use.

Firstly, for this two-asset guide example, we create a set $I = \left \{ 0, 1, 2\right \}$ of numbers ranging from 0 to m (number of assets). We can then create another set $I^{n-k} = I^2$, containing $(n-k) \times 1$ matricies of all combinations of numbes within $I$ in lexicographic order:

$$
I^2 = \left\{ \left( \begin{matrix} 0 \\ 0 \end{matrix} \right), \left( \begin{matrix} 0 \\ 1 \end{matrix} \right), \left( \begin{matrix} 0 \\ 2 \end{matrix} \right), \left( \begin{matrix} 1 \\ 0 \end{matrix} \right), \left( \begin{matrix} 1 \\ 1 \end{matrix} \right), \left( \begin{matrix} 1 \\ 2 \end{matrix} \right), \left( \begin{matrix} 2 \\ 0 \end{matrix} \right), \left( \begin{matrix} 2 \\ 1 \end{matrix} \right), \left( \begin{matrix} 2 \\ 2 \end{matrix} \right) \right\}.
$$

In [23]:
set_i = list(range(asset_count + 1))
I_factor = time_count - selected_time

set_I = list(itertools.product(set_i, repeat=I_factor))
print(set_I)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


What is worth mentioning below are the "coefficients" $b_i$. These coefficients are such that $b_0 = 1$, and $b_i = \frac{R-D_i}{U_i - D_i}$. $b_0 \geq b_1 \geq \dots \geq b_m$ must hold at all times. In our example, we have 

$$
b_0 = 1,\ \  b_1 = \frac{2}{3},\ \  b_2 = \frac{1}{2}
$$

Because these coefficients have to be organised in descending order, we will rearrange our assets and their respective factors as necessary.

In [24]:
combined = [(i, (risk_free_rate - d) / (u - d) if (u - d) else 0, s, u, d)
            for i, (s, u, d) in enumerate(zip(initial_prices, up_factors, down_factors))]

combined.insert(0, (0, 1, 0, 0, 0))
combined.sort(key=lambda x: x[1], reverse=True)
indices, coefficients, initial_prices, up_factors, down_factors = map(list, zip(*combined))

initial_prices.pop(0)
up_factors.pop(0)
down_factors.pop(0)

print("Sorted coefficients:", coefficients)
print("Sorted prices:", initial_prices)
print("Sorted up factors:", up_factors)
print("Sorted down factors:", down_factors)

Sorted coefficients: [1, 0.6666666666666664, 0.4999999999999997]
Sorted prices: [100, 100]
Sorted up factors: [1.1, 1.1]
Sorted down factors: [0.8, 0.9]


For our two-asset example, we know the set of all outcomes $\Omega = \left\{ \binom{0}{0}, \binom{0}{1}, \binom{1}{0}, \binom{1}{1} \right\}$. Here, we assign each event $\mu_i$ to each outcome $\omega$ such that

$$
\mu_0 = \begin{pmatrix}
0 \\
0 \\
\vdots \\
0
\end{pmatrix}, \quad
\mu_1 = \begin{pmatrix}
1 \\
0 \\
\vdots \\
0
\end{pmatrix}, \quad
\mu_2 = \begin{pmatrix}
1 \\
1 \\
\vdots \\
0
\end{pmatrix}, \ldots, \quad
\mu_m = \begin{pmatrix}
1 \\
1 \\
\vdots \\
1
\end{pmatrix}
$$

where each subsequent $\mu$ converts the next 0 into a 1. In our case, we have

$$
\mu_0 = \begin{pmatrix}
0 \\
0
\end{pmatrix}, \quad
\mu_1 = \begin{pmatrix}
1 \\
0
\end{pmatrix}, \quad
\mu_2 = \begin{pmatrix}
1 \\
1
\end{pmatrix}
$$

In [27]:
mu = [(1,) * i + (0,) * (asset_count - i) for i in range(asset_count + 1)]
print(mu)

[(0, 0), (1, 0), (1, 1)]


We then assign probabilities to each event in such a way that

$$
P(\mu_0) = b_0 - b_1, \quad P(\mu_1) = b_1 - b_2, \ldots, \quad P(\mu_m) = b_m
$$

Notice that if the coefficients $b_i$ were not arranged in descending order, we may have negative probailities. In our case, we have

$$
P(\mu_0) = \frac{1}{3}, \ \ P(\mu_1) = \frac{1}{6}, \ \ P(\mu_2) = \frac{1}{2}
$$

In [28]:
mu_probabilities = [coefficients[i] - coefficients[i+1] for i in range(len(coefficients)-1)] + [coefficients[-1]]
print(mu_probabilities)

[0.3333333333333336, 0.16666666666666669, 0.4999999999999997]


Now, we can combine the probabilites above, with the set $I^2$ created earlier. Each item in $I^2$ informs us of how the next step should be carried out. Take $\begin{pmatrix}
2 \\
1
\end{pmatrix}$, for example. We know that we will be working with $\mu_1$ and $\mu_2$, and $P(\mu_1)$ and $P(\mu_2)$. We can then assemble

$$
P(\mu_2) \cdot P(\mu_1) \cdot X \begin{pmatrix} 1&1 \\ 1&0 \end{pmatrix}
$$

where $\begin{pmatrix} 1&1 \\ 1&0 \end{pmatrix}$ is the payoff of the outcome combining $\mu_2$ and $\mu_1$. Some (often most) payoffs are zero. In this instance, the payoff is simply calculated with

$$
\max \left\{ \frac{(100 \times 1.1 \times 1.1) + (100 \times 1.1 \times 0.9)}{2} - 100, 0 \right\} = 10
$$

Thus, the expected payoff is $\frac{1}{6} \times \frac{1}{2} \times 10 = £0.83$. Because there are nine items within $I^2$, we know that we will be working with nine different payoffs.

In [29]:
all_matrices = [np.column_stack([mu[i] for i in indices]) for indices in set_I]

for idx, matrix in enumerate(all_matrices):
    print(f"X {idx + 1}:\n {matrix}\n")

X 1:
 [[0 0]
 [0 0]]

X 2:
 [[0 1]
 [0 0]]

X 3:
 [[0 1]
 [0 1]]

X 4:
 [[1 0]
 [0 0]]

X 5:
 [[1 1]
 [0 0]]

X 6:
 [[1 1]
 [0 1]]

X 7:
 [[1 0]
 [1 0]]

X 8:
 [[1 1]
 [1 0]]

X 9:
 [[1 1]
 [1 1]]



In [14]:
payoffs = []
for matrix in all_matrices:
    S_t = initial_prices.copy()
    for t in range(len(matrix[0])):
        for i in range(asset_count):
            multiplier = up_factors[i] if matrix[i][t] == 1 else down_factors[i]
            S_t[i] *= multiplier
    payoff = max((sum(S_t) / asset_count) - strike_price, 0) if option_type == "call" else max(strike_price - (sum(S_t) / asset_count), 0)
    payoffs.append(payoff)

print(payoffs)

[0, 0, 0, 0, 1.0000000000000142, 10.000000000000028, 0, 10.000000000000028, 21.00000000000003]


Finally, we can calculate $C_{\textnormal{max}}(0)$, our price. The full equation for our example is

$$
\begin{aligned}
c_{\max}(0) = & \, P(\mu_0)^2 \cdot X \begin{pmatrix}
0 & 0 \\
0 & 0
\end{pmatrix} + P(\mu_0) \cdot P(\mu_1) \cdot X \begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix} + P(\mu_0) \cdot P(\mu_2) \cdot X \begin{pmatrix}
0 & 1 \\
0 & 1
\end{pmatrix} \\
& + P(\mu_1) \cdot P(\mu_0) \cdot X \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix} + P(\mu_1)^2 \cdot X \begin{pmatrix}
1 & 0 \\
1 & 0
\end{pmatrix} + P(\mu_1) \cdot P(\mu_2) \cdot X \begin{pmatrix}
1 & 0 \\
1 & 1
\end{pmatrix} \\
& + P(\mu_2) \cdot P(\mu_0) \cdot X \begin{pmatrix}
1 & 1 \\
0 & 0
\end{pmatrix} + P(\mu_2) \cdot P(\mu_1) \cdot X \begin{pmatrix}
1 & 1 \\
1 & 0
\end{pmatrix} + P(\mu_2)^2 \cdot X \begin{pmatrix}
1 & 1 \\
1 & 1
\end{pmatrix}.
\end{aligned}
$$

$$
\Longrightarrow C_{\textnormal{max}}(0) = £6.94
$$

In [31]:
c_max = sum([payoffs[idx] * np.prod([mu_probabilities[i] for i in indices]) for idx, indices in enumerate(set_I)])

print(f"Price of option at time 0:", "\n\tC_max = £", round(c_max, 2))

Price of option at time 0: 
	C_max = £ 6.94
